In [62]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
from recipe_scrapers import scrape_me
import warnings
warnings.filterwarnings('ignore')

# Add favourite recipes
loved_recipes = ["https://www.gousto.co.uk/cookbook/recipes/chicken-breast-pathia-with-turmeric-rice-and-naan","https://www.gousto.co.uk/cookbook/chicken-recipes/quick-shredded-hoisin-chicken-wraps",
                "https://www.gousto.co.uk/cookbook/chicken-recipes/chilli-chicken-cashew-noodles","https://www.gousto.co.uk/cookbook/lamb-recipes/moor-ish-lamb-sweet-potato-tagine",
                "https://www.gousto.co.uk/cookbook/fish-recipes/goan-fish-spinach-curry-coriander-rice","https://www.gousto.co.uk/cookbook/beef-recipes/ultimate-cheeseburger-with-bacon-rosemary-fries"
                ,"https://www.gousto.co.uk/cookbook/recipes/korean-fried-chicken-with-yang-nyum-sauce","https://www.gousto.co.uk/cookbook/recipes/paprika-paneer-with-tomato-coconut-dal",
                "https://www.gousto.co.uk/cookbook/recipes/simply-perfect-lean-beef-spag-bol","https://www.gousto.co.uk/cookbook/recipes/char-siu-style-pork-bao-buns-with-mooli-slaw-loaded-fries",
                "https://www.gousto.co.uk/cookbook/recipes/smoky-lean-beef-enchiladas-with-fresh-tomato-salsa"]

#### Breaking down recipes into dataframe

In [63]:
# Create an empty DataFrame
recipe_df = pd.DataFrame(columns=['Title', 'Image', 'Time','Ingredients', 'Instructions', 'Instructions List', 'Yields'])

for meal in loved_recipes:
    scraper = scrape_me(meal)
    title = scraper.title()
    time = scraper.total_time()
    image = scraper.image()
    ingredients = scraper.ingredients()
    instructions = scraper.instructions()
    instructions_list = scraper.instructions_list()
    yields = scraper.yields()

    
    recipe_df = recipe_df.append({'Title': title,
                                   'Image': image,
                                   'Time': time,
                                   'Ingredients':ingredients,
                                   'Instructions': instructions,
                                   'Instructions List': instructions_list,
                                   'Yields': yields}, ignore_index=True)


In [64]:
recipe_df.head()

,Title,Image,Time,Ingredients,Instructions,Instructions List,Yields
0,Chicken Pathia With Turmeric Rice And Naan,https://production-media.gousto.co.uk/cms/mood...,35,"[32g tomato paste, 1/2 tsp cayenne pepper, 1/2...","Take your chicken out of the fridge, open the ...","[Take your chicken out of the fridge, open the...",4 servings
1,Shredded Hoisin Chicken Wraps,https://production-media.gousto.co.uk/cms/mood...,30,"[40g hoisin sauce, 2 spring onions, 1/2 cucumb...","Before you start cooking, take your chicken ou...","[Before you start cooking, take your chicken o...",4 servings
2,Chilli Chicken Thigh Cashew Noodles,https://production-media.gousto.co.uk/cms/mood...,10,"[1 tsp dried chilli flakes, 10ml tomato ketchu...",Before you begin...Get all your ingredients an...,[Before you begin...Get all your ingredients a...,4 servings
3,Moroccan-Style Lamb & Sweet Potato Tagine,https://production-media.gousto.co.uk/cms/mood...,35,"[1 tbsp ras el hanout, 11g chicken stock mix, ...",Peel and chop the brown onion[s] roughlyChop t...,[Peel and chop the brown onion[s] roughlyChop ...,4 servings
4,Goan-Style Basa & Spinach Curry With Coriander...,https://production-media.gousto.co.uk/cms/mood...,30,"[1 tbsp curry powder, 15g fresh root ginger, 8...",Boil half a kettlePeel and finely dice the red...,[Boil half a kettlePeel and finely dice the re...,4 servings


#### Creating Dash App to select meals, output is ingredients and instructions

In [65]:
# 
app = dash.Dash(__name__)

# Create a list of available meal options
meal_options = [{'label': title, 'value': title} for title in recipe_df['Title']]

# Define the layout
app.layout = html.Div([
    html.H1('Recipe Ingredients and Instructions'),
    dcc.Dropdown(
        id='meal-dropdown',
        options=meal_options,
        placeholder='Select meal(s)',
        multi=True
    ),
    html.Div(id='output-ingredients'),
    html.Div(id='output-instructions')
])

# Define the callback function for ingredients
@app.callback(
    Output('output-ingredients', 'children'),
    [Input('meal-dropdown', 'value')]
)
def display_ingredients(selected_meals):
    if selected_meals:
        ingredients = []
        for meal in selected_meals:
            meal_ingredients = recipe_df.loc[recipe_df['Title'] == meal, 'Ingredients'].values[0]
            meal_ingredients_formatted = ', '.join(meal_ingredients)
            ingredients.append(html.Div([
                html.H3(f'Ingredients for {meal}:'),
                dcc.Markdown(meal_ingredients_formatted)
            ]))
        return ingredients
    else:
        return html.Div()

# Define the callback function for instructions
@app.callback(
    Output('output-instructions', 'children'),
    [Input('meal-dropdown', 'value')]
)
def display_instructions(selected_meals):
    if selected_meals:
        instructions = []
        for meal in selected_meals:
            meal_instructions = recipe_df.loc[recipe_df['Title'] == meal, 'Instructions List'].values[0]
            meal_instructions_formatted = '\n'.join(meal_instructions)
            instructions.append(html.Div([
                html.H3(f'Instructions for {meal}:'),
                dcc.Markdown(meal_instructions_formatted)
            ]))
        return instructions
    else:
        return html.Div()

# Run the app
if __name__ == '__main__':
    app.run_server()
    
# Click on link to open up WebApp

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
